# MCTS + Whittle Indices Experiments

Analyze the performance of various algorithms to solve the joint matching + activity task, when the number of volunteers is large and structured

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import random 
import matplotlib.pyplot as plt
import json 
import argparse 
import sys
import secrets

In [88]:
from rmab.simulator import RMABSimulator, run_heterogenous_policy, get_discounted_reward
from rmab.omniscient_policies import *
from rmab.fr_dynamics import get_all_transitions, get_db_data 
from rmab.compute_whittle import arm_compute_whittle_multi_prob
from rmab.mcts_policies import *
from rmab.utils import get_save_path, delete_duplicate_results, create_prob_distro
import resource

In [4]:
torch.cuda.set_per_process_memory_fraction(0.5)
torch.set_num_threads(1)
resource.setrlimit(resource.RLIMIT_AS, (30 * 1024 * 1024 * 1024, -1))

In [5]:
is_jupyter = 'ipykernel' in sys.modules

In [167]:
if is_jupyter: 
    seed        = 42
    n_arms      = 10
    volunteers_per_arm = 1
    budget      = 10
    discount    = 0.9
    alpha       = 3 
    n_episodes  = 1
    episode_len = 50 
    n_epochs    = 1
    save_with_date = False 
    TIME_PER_RUN = 0.01 * 1000
    lamb = 0
    prob_distro = 'food_rescue'
    reward_type = "probability"
    reward_parameters = {'universe_size': 20, 'arm_set_low': 8, 'arm_set_high': 10}
    policy_lr=5e-3
    value_lr=1e-4
    train_iterations = 30
    test_iterations = 30
    out_folder = 'iterative'
else:
    parser = argparse.ArgumentParser()
    parser.add_argument('--n_arms',         '-N', help='num beneficiaries (arms)', type=int, default=2)
    parser.add_argument('--volunteers_per_arm',         '-V', help='volunteers per arm', type=int, default=5)
    parser.add_argument('--episode_len',    '-H', help='episode length', type=int, default=50)
    parser.add_argument('--n_episodes',     '-T', help='num episodes', type=int, default=1)
    parser.add_argument('--budget',         '-B', help='budget', type=int, default=3)
    parser.add_argument('--n_epochs',       '-E', help='number of epochs (num_repeats)', type=int, default=1)
    parser.add_argument('--discount',       '-d', help='discount factor', type=float, default=0.9)
    parser.add_argument('--alpha',          '-a', help='alpha: for conf radius', type=float, default=3)
    parser.add_argument('--lamb',          '-l', help='lambda for matching-engagement tradeoff', type=float, default=0.5)
    parser.add_argument('--universe_size', help='For set cover, total num unvierse elems', type=int, default=10)
    parser.add_argument('--arm_set_low', help='Least size of arm set, for set cover', type=float, default=3)
    parser.add_argument('--arm_set_high', help='Largest size of arm set, for set cover', type=float, default=6)
    parser.add_argument('--reward_type',          '-r', help='Which type of custom reward', type=str, default='set_cover')
    parser.add_argument('--seed',           '-s', help='random seed', type=int, default=42)
    parser.add_argument('--prob_distro',           '-p', help='which prob distro [uniform,uniform_small,uniform_large,normal]', type=str, default='uniform')
    parser.add_argument('--time_per_run',      '-t', help='time per MCTS run', type=float, default=.01*1000)
    parser.add_argument('--policy_lr', help='Learning Rate Policy', type=float, default=5e-3)
    parser.add_argument('--value_lr', help='Learning Rate Value', type=float, default=1e-4)
    parser.add_argument('--train_iterations', help='Number of MCTS train iterations', type=int, default=30)
    parser.add_argument('--test_iterations', help='Number of MCTS test iterations', type=int, default=30)
    parser.add_argument('--out_folder', help='Which folder to write results to', type=str, default='iterative')

    parser.add_argument('--use_date', action='store_true')

    args = parser.parse_args()

    n_arms      = args.n_arms
    volunteers_per_arm = args.volunteers_per_arm
    budget      = args.budget
    discount    = args.discount
    alpha       = args.alpha 
    seed        = args.seed
    n_episodes  = args.n_episodes
    episode_len = args.episode_len
    n_epochs    = args.n_epochs
    lamb = args.lamb
    save_with_date = args.use_date
    TIME_PER_RUN = args.time_per_run
    prob_distro = args.prob_distro
    policy_lr = args.policy_lr 
    value_lr = args.value_lr 
    out_folder = args.out_folder
    train_iterations = args.train_iterations 
    test_iterations = args.test_iterations 
    reward_type = args.reward_type
    reward_parameters = {'universe_size': args.universe_size,
                        'arm_set_low': args.arm_set_low, 
                        'arm_set_high': args.arm_set_high}

save_name = secrets.token_hex(4)  

In [168]:
n_states = 2
n_actions = 2

In [169]:
all_population_size = 100 
all_transitions = get_all_transitions(all_population_size)

In [170]:
if prob_distro == "food_rescue":
    probs_by_user = json.load(open("../../results/food_rescue/match_probs.json","r"))
    donation_id_to_latlon, recipient_location_to_latlon, rescues_by_user, all_rescue_data, user_id_to_latlon = get_db_data()
    probs_by_num = {}
    for i in rescues_by_user:
        if str(i) in probs_by_user and probs_by_user[str(i)] > 0:
            if len(rescues_by_user[i]) not in probs_by_num:
                probs_by_num[len(rescues_by_user[i])] = []
            probs_by_num[len(rescues_by_user[i])].append(probs_by_user[str(i)])

In [171]:
def create_environment(seed):
    random.seed(seed)
    np.random.seed(seed)

    all_features = np.arange(all_population_size)
    N = all_population_size*volunteers_per_arm
    if reward_type == "set_cover":
        set_sizes = [np.random.poisson(int(reward_parameters['arm_set_low'])) for i in range(N)]
        match_probabilities = [set([np.random.randint(0,reward_parameters['universe_size']) for _ in range(set_sizes[i])]) for i in range(N)]
    elif prob_distro == "food_rescue":
        match_probabilities = [np.random.choice(probs_by_num[(i+2*volunteers_per_arm)//volunteers_per_arm]) for i in range(N)] 
    else:
        match_probabilities = [np.random.uniform(reward_parameters['arm_set_low'],reward_parameters['arm_set_high']) for i in range(N)]

    simulator = RMABSimulator(all_population_size, all_features, all_transitions,
                n_arms, volunteers_per_arm, episode_len, n_epochs, n_episodes, budget, discount,number_states=n_states, reward_style='custom',match_probability_list=match_probabilities,TIME_PER_RUN=TIME_PER_RUN)
    simulator.reward_type = reward_type 
    simulator.reward_parameters = reward_parameters 
    return simulator 

In [172]:
def run_multi_seed(seed_list,policy,is_mcts=False,per_epoch_function=None,train_iterations=0,test_iterations=0,test_length=20):
    memories = []
    scores = {
        'reward': [],
        'time': [], 
        'match': [], 
        'active_rate': [],
    }

    for seed in seed_list:
        simulator = create_environment(seed)

        if is_mcts:
            simulator.mcts_train_iterations = train_iterations
            simulator.mcts_test_iterations = test_iterations
            simulator.policy_lr = policy_lr
            simulator.value_lr = value_lr

        if is_mcts:
            match, active_rate, memory = run_heterogenous_policy(simulator, n_episodes, n_epochs, discount,policy,seed,lamb=lamb,should_train=True,test_T=test_length,get_memory=True,per_epoch_function=per_epoch_function)
        else:
            match, active_rate = run_heterogenous_policy(simulator, n_episodes, n_epochs, discount,policy,seed,lamb=lamb,should_train=True,test_T=test_length,per_epoch_function=per_epoch_function)
        time_whittle = simulator.time_taken
        discounted_reward = get_discounted_reward(match,active_rate,discount,lamb)
        scores['reward'].append(discounted_reward)
        scores['time'].append(time_whittle)
        scores['match'].append(np.mean(match))
        scores['active_rate'].append(np.mean(active_rate))
        if is_mcts:
            memories.append(memory)

    return scores, memories, simulator

In [173]:
results = {}
results['parameters'] = {'seed'      : seed,
        'n_arms'    : n_arms,
        'volunteers_per_arm': volunteers_per_arm, 
        'budget'    : budget,
        'discount'  : discount, 
        'alpha'     : alpha, 
        'n_episodes': n_episodes, 
        'episode_len': episode_len, 
        'n_epochs'  : n_epochs, 
        'lamb': lamb,
        'time_per_run': TIME_PER_RUN, 
        'prob_distro': prob_distro, 
        'policy_lr': policy_lr, 
        'value_lr': value_lr, 
        'reward_type': reward_type, 
        'universe_size': reward_parameters['universe_size'],
        'arm_set_low': reward_parameters['arm_set_low'], 
        'arm_set_high': reward_parameters['arm_set_high'],
        } 

## Index Policies

In [164]:
seed_list = [seed]

In [145]:
policy = greedy_policy
name = "greedy"

rewards, memory, simulator = run_multi_seed(seed_list,policy,test_length=n_episodes*episode_len)
results['{}_reward'.format(name)] = rewards['reward']
results['{}_match'.format(name)] =  rewards['match'] 
results['{}_active'.format(name)] = rewards['active_rate']
results['{}_time'.format(name)] =  rewards['time']
print(np.mean(rewards['reward']))

acting should always be good! (0, 1) 0.108 < 0.183
good start state should always be good! 0.380 < 0.508
good start state should always be good! 0.506 < 0.760
cohort [58 53 90 86 52 57 37 15 59 23 30 75 39 20 96 48 17 16 25 55 85  9 18 77
 99 66 84 95 78 49 74 69 76 70 73 83 24 45 97 63 19 54 82  5 56 67  1 89
 21 88 65 12 35 28 42 68 44 38 81 79 29  3 31 60 51 98 50 41 14  8 26 13
 91  0  2 46 64 93 43 36 61 22 47 92 33 11 71 72  6 27 40  4 32 94 34  7
 80 10 62 87]
Took 0.22308754920959473 time for inference and 0.00769495964050293 time for training
4.531325132194763


In [146]:
policy = random_policy
name = "random"

rewards, memory, simulator = run_multi_seed(seed_list,policy,test_length=n_episodes*episode_len)
results['{}_reward'.format(name)] = rewards['reward']
results['{}_match'.format(name)] =  rewards['match'] 
results['{}_active'.format(name)] = rewards['active_rate']
results['{}_time'.format(name)] =  rewards['time']
print(np.mean(rewards['reward']))

acting should always be good! (0, 1) 0.108 < 0.183
good start state should always be good! 0.380 < 0.508
good start state should always be good! 0.506 < 0.760
cohort [58 53 90 86 52 57 37 15 59 23 30 75 39 20 96 48 17 16 25 55 85  9 18 77
 99 66 84 95 78 49 74 69 76 70 73 83 24 45 97 63 19 54 82  5 56 67  1 89
 21 88 65 12 35 28 42 68 44 38 81 79 29  3 31 60 51 98 50 41 14  8 26 13
 91  0  2 46 64 93 43 36 61 22 47 92 33 11 71 72  6 27 40  4 32 94 34  7
 80 10 62 87]
Took 0.07633495330810547 time for inference and 0.0016407966613769531 time for training
2.840252551919876


In [147]:
policy = whittle_activity_policy
name = "whittle_activity"

rewards, memory, simulator = run_multi_seed(seed_list,policy,test_length=n_episodes*episode_len)
results['{}_reward'.format(name)] = rewards['reward']
results['{}_match'.format(name)] =  rewards['match'] 
results['{}_active'.format(name)] = rewards['active_rate']
results['{}_time'.format(name)] =  rewards['time']
print(np.mean(rewards['reward']))

acting should always be good! (0, 1) 0.108 < 0.183
good start state should always be good! 0.380 < 0.508
good start state should always be good! 0.506 < 0.760
cohort [58 53 90 86 52 57 37 15 59 23 30 75 39 20 96 48 17 16 25 55 85  9 18 77
 99 66 84 95 78 49 74 69 76 70 73 83 24 45 97 63 19 54 82  5 56 67  1 89
 21 88 65 12 35 28 42 68 44 38 81 79 29  3 31 60 51 98 50 41 14  8 26 13
 91  0  2 46 64 93 43 36 61 22 47 92 33 11 71 72  6 27 40  4 32 94 34  7
 80 10 62 87]
Took 4.527488708496094 time for inference and 4.490064382553101 time for training
3.1156738053066375


In [148]:
policy = whittle_policy
name = "linear_whittle"

rewards, memory, simulator = run_multi_seed(seed_list,policy,test_length=n_episodes*episode_len)
results['{}_reward'.format(name)] = rewards['reward']
results['{}_match'.format(name)] =  rewards['match'] 
results['{}_active'.format(name)] = rewards['active_rate']
results['{}_time'.format(name)] =  rewards['time']
print(np.mean(rewards['reward']))

acting should always be good! (0, 1) 0.108 < 0.183
good start state should always be good! 0.380 < 0.508
good start state should always be good! 0.506 < 0.760
cohort [58 53 90 86 52 57 37 15 59 23 30 75 39 20 96 48 17 16 25 55 85  9 18 77
 99 66 84 95 78 49 74 69 76 70 73 83 24 45 97 63 19 54 82  5 56 67  1 89
 21 88 65 12 35 28 42 68 44 38 81 79 29  3 31 60 51 98 50 41 14  8 26 13
 91  0  2 46 64 93 43 36 61 22 47 92 33 11 71 72  6 27 40  4 32 94 34  7
 80 10 62 87]
Took 5.185845851898193 time for inference and 5.680272340774536 time for training
5.5014195523459835


In [149]:
policy = whittle_greedy_policy
name = "greedy_whittle"

rewards, memory, simulator = run_multi_seed(seed_list,policy,test_length=n_episodes*episode_len)
results['{}_reward'.format(name)] = rewards['reward']
results['{}_match'.format(name)] =  rewards['match'] 
results['{}_active'.format(name)] = rewards['active_rate']
results['{}_time'.format(name)] =  rewards['time']
print(np.mean(rewards['reward']))

acting should always be good! (0, 1) 0.108 < 0.183
good start state should always be good! 0.380 < 0.508
good start state should always be good! 0.506 < 0.760
cohort [58 53 90 86 52 57 37 15 59 23 30 75 39 20 96 48 17 16 25 55 85  9 18 77
 99 66 84 95 78 49 74 69 76 70 73 83 24 45 97 63 19 54 82  5 56 67  1 89
 21 88 65 12 35 28 42 68 44 38 81 79 29  3 31 60 51 98 50 41 14  8 26 13
 91  0  2 46 64 93 43 36 61 22 47 92 33 11 71 72  6 27 40  4 32 94 34  7
 80 10 62 87]
Took 4.63590669631958 time for inference and 4.89432168006897 time for training
5.466044041940122


In [150]:
if n_arms * volunteers_per_arm <= 4:
    policy = q_iteration_policy
    per_epoch_function = q_iteration_custom_epoch()
    name = "optimal"

    rewards, memory, simulator = run_multi_seed(seed_list,policy,per_epoch_function=per_epoch_function,test_length=n_episodes*episode_len)
    results['{}_reward'.format(name)] = rewards['reward']
    results['{}_match'.format(name)] =  rewards['match'] 
    results['{}_active'.format(name)] = rewards['active_rate']
    results['{}_time'.format(name)] =  rewards['time']
    print(np.mean(rewards['reward']))

In [151]:
policy = shapley_whittle_custom_policy 
name = "shapley_whittle_custom"

rewards, memory, simulator = run_multi_seed(seed_list,policy,test_length=n_episodes*episode_len)
results['{}_reward'.format(name)] = rewards['reward']
results['{}_match'.format(name)] =  rewards['match'] 
results['{}_active'.format(name)] = rewards['active_rate']
results['{}_time'.format(name)] =  rewards['time']
print(np.mean(rewards['reward']))

acting should always be good! (0, 1) 0.108 < 0.183
good start state should always be good! 0.380 < 0.508
good start state should always be good! 0.506 < 0.760
cohort [58 53 90 86 52 57 37 15 59 23 30 75 39 20 96 48 17 16 25 55 85  9 18 77
 99 66 84 95 78 49 74 69 76 70 73 83 24 45 97 63 19 54 82  5 56 67  1 89
 21 88 65 12 35 28 42 68 44 38 81 79 29  3 31 60 51 98 50 41 14  8 26 13
 91  0  2 46 64 93 43 36 61 22 47 92 33 11 71 72  6 27 40  4 32 94 34  7
 80 10 62 87]
Took 5.16560435295105 time for inference and 7.733052730560303 time for training
5.5014195523459835


In [152]:
policy = mcts_shapley_policy
name = "mcts_shapley"

rewards, memory, simulator = run_multi_seed(seed_list,policy,is_mcts=True,test_iterations=400,test_length=n_episodes*episode_len)
results['{}_reward'.format(name)] = rewards['reward']
results['{}_match'.format(name)] =  rewards['match'] 
results['{}_active'.format(name)] = rewards['active_rate']
results['{}_time'.format(name)] =  rewards['time']
np.mean(rewards['reward'])

acting should always be good! (0, 1) 0.108 < 0.183
good start state should always be good! 0.380 < 0.508
good start state should always be good! 0.506 < 0.760
cohort [58 53 90 86 52 57 37 15 59 23 30 75 39 20 96 48 17 16 25 55 85  9 18 77
 99 66 84 95 78 49 74 69 76 70 73 83 24 45 97 63 19 54 82  5 56 67  1 89
 21 88 65 12 35 28 42 68 44 38 81 79 29  3 31 60 51 98 50 41 14  8 26 13
 91  0  2 46 64 93 43 36 61 22 47 92 33 11 71 72  6 27 40  4 32 94 34  7
 80 10 62 87]
Took 32.642507791519165 time for inference and 8.940064668655396 time for training


5.5014195523459835

In [175]:
if n_arms * volunteers_per_arm <= 10:
    policy = whittle_iterative_policy
    name = "iterative_whittle"

    rewards, memory, simulator = run_multi_seed(seed_list,policy,test_length=n_episodes*episode_len)
    results['{}_reward'.format(name)] = rewards['reward']
    results['{}_match'.format(name)] =  rewards['match'] 
    results['{}_active'.format(name)] = rewards['active_rate']
    results['{}_time'.format(name)] =  rewards['time']
    np.mean(rewards['reward'])

acting should always be good! (0, 1) 0.108 < 0.183
good start state should always be good! 0.380 < 0.508
good start state should always be good! 0.506 < 0.760
cohort [58 53 90 86 52 57 37 15 59 23]
Took 80.4186794757843 time for inference and 4.843940019607544 time for training


In [156]:
policy = greedy_whittle_iterative_policy
name = "greedy_iterative_whittle"

rewards, memory, simulator = run_multi_seed(seed_list,policy,test_iterations=400,test_length=n_episodes*episode_len)
results['{}_reward'.format(name)] = rewards['reward']
results['{}_match'.format(name)] =  rewards['match'] 
results['{}_active'.format(name)] = rewards['active_rate']
results['{}_time'.format(name)] =  rewards['time']
np.mean(rewards['reward'])

acting should always be good! (0, 1) 0.108 < 0.183
good start state should always be good! 0.380 < 0.508
good start state should always be good! 0.506 < 0.760
cohort [58 53 90 86 52 57 37 15 59 23 30 75 39 20 96 48 17 16 25 55 85  9 18 77
 99 66 84 95 78 49 74 69 76 70 73 83 24 45 97 63 19 54 82  5 56 67  1 89
 21 88 65 12 35 28 42 68 44 38 81 79 29  3 31 60 51 98 50 41 14  8 26 13
 91  0  2 46 64 93 43 36 61 22 47 92 33 11 71 72  6 27 40  4 32 94 34  7
 80 10 62 87]
Took 12.557528018951416 time for inference and 5.503240585327148 time for training


5.466044041940122

In [176]:
if n_arms * volunteers_per_arm <= 10:
    policy = shapley_whittle_iterative_policy
    name = "shapley_iterative_whittle"

    rewards, memory, simulator = run_multi_seed(seed_list,policy,test_iterations=400,test_length=n_episodes*episode_len)
    results['{}_reward'.format(name)] = rewards['reward']
    results['{}_match'.format(name)] =  rewards['match'] 
    results['{}_active'.format(name)] = rewards['active_rate']
    results['{}_time'.format(name)] =  rewards['time']
    np.mean(rewards['reward'])

acting should always be good! (0, 1) 0.108 < 0.183
good start state should always be good! 0.380 < 0.508
good start state should always be good! 0.506 < 0.760
cohort [58 53 90 86 52 57 37 15 59 23]


/usr0/home/naveenr/miniconda3/envs/food/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr0/home/naveenr/miniconda3/envs/food/lib/python3.8/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Took 17.459863901138306 time for inference and 4.572295665740967 time for training


## Write Data

In [20]:
save_path = get_save_path(out_folder,save_name,seed,use_date=save_with_date)

In [21]:
delete_duplicate_results(out_folder,"",results)

In [22]:
json.dump(results,open('../../results/'+save_path,'w'))